In [34]:
## Movie Recommonder System
############################
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
df_ratings=pd.read_csv('ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [36]:
df_movie=pd.read_csv('movies.csv')
df_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [37]:
df_movie_ratings=pd.merge(df_ratings,df_movie,on='movieId')
df_movie_ratings.drop(['timestamp','genres'],axis=1,inplace=True)

In [38]:
df_movie_ratings

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)
...,...,...,...,...
100831,610,160341,2.5,Bloodmoon (1997)
100832,610,160527,4.5,Sympathy for the Underdog (1971)
100833,610,160836,3.0,Hazard (2005)
100834,610,163937,3.5,Blair Witch (2016)


In [39]:
# Creating user_id verses Movie Matrix
#######################################
movie_mat=df_movie_ratings.pivot_table(index=['userId'],columns=['title'],values=['rating'])
movie_mat.head()

rating                                          \
title  '71 (2014) 'Hellboy': The Seeds of Creation (2004)   
userId                                                      
1             NaN                                     NaN   
2             NaN                                     NaN   
3             NaN                                     NaN   
4             NaN                                     NaN   
5             NaN                                     NaN   

                                                                             \
title  'Round Midnight (1986) 'Salem's Lot (2004) 'Til There Was You (1997)   
userId                                                                        
1                         NaN                 NaN                       NaN   
2                         NaN                 NaN                       NaN   
3                         NaN                 NaN                       NaN   
4                         NaN                 NaN                       NaN   
5                         NaN                 NaN                       NaN   

                                                           \
title  'Tis the Season for Love (2015) 'burbs, The (1989)   
userId                                                      
1                                  NaN                NaN   
2                                  NaN                NaN   
3                                  NaN                NaN   
4                                  NaN                NaN   
5                                  NaN                NaN   

                                                         \
title  'night Mother (1986) (500) Days of Summer (2009)   
userId                                                    
1                       NaN                         NaN   
2                       NaN                         NaN   
3                       NaN                         NaN   
4                       NaN                         NaN   
5                       NaN                         NaN   

                                       ...                           \
title  *batteries not included (1987)  ... Zulu (2013) [REC] (2007)   
userId                                 ...                            
1                                 NaN  ...         NaN          NaN   
2                                 NaN  ...         NaN          NaN   
3                                 NaN  ...         NaN          NaN   
4                                 NaN  ...         NaN          NaN   
5                                 NaN  ...         NaN          NaN   

                                              \
title  [REC]² (2009) [REC]³ 3 Génesis (2012)   
userId                                         
1                NaN                     NaN   
2                NaN                     NaN   
3                NaN                     NaN   
4                NaN                     NaN   
5                NaN                     NaN   

                                                                               \
title  anohana: The Flower We Saw That Day - The Movie (2013) eXistenZ (1999)   
userId                                                                          
1                                                     NaN                 NaN   
2                                                     NaN                 NaN   
3                                                     NaN                 NaN   
4                                                     NaN                 NaN   
5                                                     NaN                 NaN   

                                                                        \
title  xXx (2002) xXx: State of the Union (2005) ¡Three Amigos! (1986)   
userId                                                                   
1             NaN                            NaN                   4.0   
2             NaN           

In [12]:
#  movie_mat.fillna(0,inplace=True)

In [20]:
# movie_mat.head()

In [40]:
print("Before: ",movie_mat.shape)
movie_mat = movie_mat.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",movie_mat.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [41]:
def standardize(row):
    new_row=(row-row.mean())/(row.max()-row.mean())
    return new_row

In [42]:
user_ratings=movie_mat.apply(standardize)

In [43]:
item_similarity=cosine_similarity(user_ratings.T)

In [44]:
item_sim_df=pd.DataFrame(item_similarity,index=user_ratings.columns,columns=user_ratings.columns)

In [45]:
type(item_sim_df)

pandas.core.frame.DataFrame

In [48]:
item_sim_df

rating  \
title                                    'burbs, The (1989)   
       title                                                  
rating 'burbs, The (1989)                          1.000000   
       (500) Days of Summer (2009)                 0.063117   
       10 Cloverfield Lane (2016)                 -0.023768   
       10 Things I Hate About You (1999)           0.143482   
       10,000 BC (2008)                            0.011998   
...                                                     ...   
       Zoolander (2001)                            0.049897   
       Zootopia (2016)                             0.003233   
       eXistenZ (1999)                             0.187953   
       xXx (2002)                                  0.062174   
       ¡Three Amigos! (1986)                       0.353194   

                                                                      \
title                                    (500) Days of Summer (2009)   
       title                                                           
rating 'burbs, The (1989)                                   0.063117   
       (500) Days of Summer (2009)                          1.000000   
       10 Cloverfield Lane (2016)                           0.142471   
       10 Things I Hate About You (1999)                    0.273989   
       10,000 BC (2008)                                     0.193960   
...                                                              ...   
       Zoolander (2001)                                     0.252226   
       Zootopia (2016)                                      0.216007   
       eXistenZ (1999)                                      0.053614   
       xXx (2002)                                           0.241092   
       ¡Three Amigos! (1986)                                0.125905   

                                                                     \
title                                    10 Cloverfield Lane (2016)   
       title                                                          
rating 'burbs, The (1989)                                 -0.023768   
       (500) Days of Summer (2009)                         0.142471   
       10 Cloverfield Lane (2016)                          1.000000   
       10 Things I Hate About You (1999)                  -0.005799   
       10,000 BC (2008)                                    0.112396   
...                                                             ...   
       Zoolander (2001)                                    0.195054   
       Zootopia (2016)                                     0.319371   
       eXistenZ (1999)                                     0.177846   
       xXx (2002)                                          0.096638   
       ¡Three Amigos! (1986)                               0.002733   

                                                                            \
title                                    10 Things I Hate About You (1999)   
       title                                                                 
rating 'burbs, The (1989)                                         0.143482   
       (500) Days of Summer (2009)                                0.273989   
       10 Cloverfield Lane (2016)                                -0.005799   
       10 Things I Hate About You (1999)                          1.000000   
       10,000 BC (2008)                                           0.244670   
...                                                                    ...   
       Zoolander (2001)                                           0.281934   
       Zootopia (2016)                                            0.050031   
       eXistenZ (1999)                                            0.121029   
       xXx (2002)                                                 0.130813   
       ¡Three Amigos! (1986)                                      0.110612   

                                                           \
title       

In [33]:
# var="101 Dalmatians (1996)"
# # a=item_sim_df.get(var)
# a

In [28]:
def get_similar_movie(movie,rating):
    similar_score=item_sim_df[movie]*(rating-2.5)
    similar_score=similar_score.sort_values(ascending=False)
    return similar_score

In [15]:
## Getting Similar Movie
# print(get_similar_movie("10,000 BC (2008",5))
    

In [29]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar_movie(movie,rating),ignore_index = True)

similar_movies.head(10)

KeyError: '(500) Days of Summer (2009)'